<a href="https://colab.research.google.com/github/ZHAbotorabi/AI-Agents/blob/main/AI_Agent_Real_Time_Stock_Market_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an AI Agent for Real-Time Stock Market Insights

## Introduction
In today's fast-paced financial landscape, timely access to data is crucial for making informed investment decisions. This article presents the development of an AI agent designed to provide users with real-time updates on stock market news, weather conditions, and current stock prices. By integrating various APIs, this agent streamlines the process of accessing essential information in one convenient platform, demonstrating the practical applications of AI in finance.

## Problem Statement
Investors often face the challenge of sifting through multiple sources to gather relevant information about stock markets and environmental conditions. Traditional methods can be time-consuming and inefficient, potentially leading to missed opportunities. This AI agent addresses these issues by consolidating crucial data—current market news, weather updates, stock prices, and time—into a single interface, enhancing the user's decision-making process.

## Design and Methodology
The AI agent is designed to be flexible and customizable, catering to the specific needs and demands of its users. It employs a multi-agent architecture that can be extended for more complex analyses and decision-making processes. The implementation is done in Python, utilizing several libraries and APIs to achieve its functionalities:

- **Libraries Used**:
  - **Requests**: For making HTTP requests to external APIs.
  - **Datetime**: For managing date and time operations.
  - **yfinance**: For fetching stock data.

- **APIs Integrated**:
  - **OpenWeatherMap API**: Provides current weather information for specified cities.
  - **NewsAPI**: Fetches the latest stock market news articles.

## Implementation/Code Setup

The AI agent consists of import statements, the setup of the llm, and several key functionalities/tools:

```python
import requests  # For making API calls to get weather and stock market news
import openai  # For interacting with OpenAI models
from langchain.chat_models import ChatOpenAI  # For chat models in LangChain
from langchain.prompts import PromptTemplate  # For creating prompts
from langchain_core.tools import tool  # For defining tools
from langchain_core.runnables import RunnableSequence  # For creating runnable sequences
import yfinance as yf  # For fetching stock market data
from datetime import datetime  # For handling date and time

# Initialize the OpenAI model
OPENAI_MODEL_4oMini = "gpt-4o-mini"
llm = ChatOpenAI(model=OPENAI_MODEL_4oMini, temperature=0, max_tokens=500)

# Define a prompt template for the agent
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are an AI agent. Respond to the user input in a helpful manner: {user_input}"
)





### 1. Current Date and Time Tool
This function retrieves the current date and time, formatted for easy readability. It serves as a basic but essential feature for users who need to track time-sensitive information.

```python
from datetime import datetime

@tool
def get_current_datetime_tool() -> str:
    """
    Tool that returns the current date and time.
    
    Returns:
        str: A string representation of the current date and time.
    """
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    return f"The current date and time is: {current_time}"


## 2. Weather Tool
This tool retrieves real-time weather information for specified cities (e.g., Calgary and Rome) using the OpenWeatherMap API. The JSON response is processed to extract and present a user-friendly weather description.

```python
import requests

@tool
def get_weather_tool(city: str) -> str:
    """Get the weather of a city."""
    api_key = "YOUR_API_KEY"  # Replace with your actual API key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()
        
        # Extract relevant information from the JSON data
        weather_description = data['weather'][0]['description']
        return f"The weather in {city} is currently {weather_description}."
    
    except requests.exceptions.RequestException as e:
        return f"Error retrieving weather data: {e}"


## 3. Stock Market Price Analysis Tool
This tool retrieves the current stock prices for major companies like Apple and Google and analyzes whether their prices have increased or decreased compared to their three-month average. This feature is crucial for investors looking to make timely decisions based on market trends.

```python
@tool
def get_stock_price_analysis(stock_symbol: str) -> str:
    """Analyze stock price trends for a given stock symbol."""
    # Logic to fetch current price and historical data to compute average
    current_price = get_current_price(stock_symbol)  # Placeholder for actual function
    three_month_average = get_three_month_average(stock_symbol)  # Placeholder for actual function
    
    if current_price > three_month_average:
        return f"The current price of {stock_symbol} is {current_price}, which is higher than its three-month average of {three_month_average}."
    elif current_price < three_month_average:
        return f"The current price of {stock_symbol} is {current_price}, which is lower than its three-month average of {three_month_average}."
    else:
        return f"The current price of {stock_symbol} is equal to its three-month average of {three_month_average}."




## 4. Stock Market News Tool
This function retrieves the latest stock market news articles using the NewsAPI.

```python
@tool
def get_stock_market_news(api_key: str) -> str:
    """Get the daily stock market news."""
    url = f"https://newsapi.org/v2/everything?q=stock%20market&language=en&sortBy=publishedAt&apiKey={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        news_data = response.json()
        
        if news_data['status'] == 'ok':
            articles = news_data['articles'][:2]  # Get the top 2 articles
            news_summary = "Today's Stock Market News:\n"
            
            for i, article in enumerate(articles, start=1):
                title = article['title']
                description = article['description']
                url = article['url']
                
                news_summary += f"{i}. {title}\n   {description}\n   Read more: {url}\n\n"
            
            return news_summary
        else:
            return "Error retrieving news data."
    
    except requests.exceptions.RequestException as e:
        return f"Error retrieving stock market news: {e}"



## 5. Stock Comparison Tool
This tool compares the current stock price to the average price over a specified period.

``` python
import yfinance as yf

@tool
def get_stock_comparison_tool(symbol: str, period: str, steps: int) -> str:
    """
    Tool for comparing the average stock price over a specified period with today's price.
    
    Args:
        symbol (str): Stock symbol to retrieve data for (e.g., "AAPL", "GOOGL").
        period (str): The period to fetch data for (must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']).
        steps (int): The number of days to calculate the average from (max 120).

    Returns:
        str: A comparison result of today's price versus the average price over the specified days.
    """
    
    # Define valid periods and enforce max step value
    valid_periods = ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    if period not in valid_periods:
        return f"Error: Invalid period. Choose one from {valid_periods}."
    
    if steps > 120:
        return "Error: The number of days (steps) cannot exceed 120."
    
    try:
        # Fetch historical stock data for the specified period
        stock = yf.Ticker(symbol)
        stock_info = stock.history(period=period)
        
        if stock_info.empty:
            return f"Could not retrieve stock data for {symbol}."
        
        # Ensure steps is within the bounds of the fetched data
        if steps >= len(stock_info):
            return f"Error: The number of days (steps) exceeds the available data for the specified period."
        
        # Calculate the average closing price over the last 'steps' days
        avg_price = stock_info['Close'].iloc[-(steps+1):-1].mean()  # Exclude today for the average
        
        # Get today's closing price (last entry in the data)
        today_price = stock_info['Close'].iloc[-1]
        
        # Compare today's price with the 'steps'-day average
        if today_price > avg_price:
            comparison = "higher (>)"
        elif today_price < avg_price:
            comparison = "lower (<)"
        else:
            comparison = "equal (=)"
        
        return (f"{symbol}'s stock price today is ${today_price:.2f}, "
                f"which is {comparison} than the average over the last {steps}-step in {period}-period (${avg_price:.2f}).")
    
    except Exception as e:
        return f"Error retrieving or processing stock data: {e}"


## 6. Initializing the Agent
The agent is initialized with the defined tools and is ready to process user requests.

``` python
agent = initialize_agent(
    tools=[get_current_datetime_tool, get_weather_tool, get_stock_market_news, get_stock_price, get_stock_comparison_tool],
    agent=AgentType.OPENAI_FUNCTIONS,    
    llm=llm,
    verbose=True,
    max_iterations=2
)


## 7. Example User Input and the Output
Here's an example of how to use the agent to retrieve information:

``` python
# Example user input to the agent
user_input = """I need the datetime, weather of Calgary and Rome, today's top 2 news of the stock market, and price of AAPL,
and I need to know the average of 50-step in a 3-monthly period of GOOGL."""
agent.run(user_input)


## Output of the Example
Here’s how the output of the agent would look in a terminal:
``` bash
> Entering new AgentExecutor chain...
Invoking: `get_current_datetime_tool` with `{}`

The current date and time is: 2024-10-05 18:47:35
Invoking: `get_weather_tool` with `{'city': 'Calgary'}`

The weather in Calgary is currently scattered clouds.
Invoking: `get_weather_tool` with `{'city': 'Rome'}`

The weather in Rome is currently clear sky.
Invoking: `get_stock_market_news` with `{'api_key': 'your_api_key'}`

Today's Stock Market News:
1. Appleton Partners Inc. MA Decreases Stock Holdings in iShares Core S&P Mid-Cap ETF (NYSEARCA:IJH)
   Appleton Partners Inc. MA lessened its stake in iShares Core S&P Mid-Cap ETF by 2.4% during the 3rd quarter.
   Read more: https://www.etfdailynews.com/2024/10/04/appleton-partners-inc-ma-decreases-stock-holdings-in-ishares-core-sp-mid-cap-etf-nysearcaijh/

2. Appleton Partners Inc. MA Decreases Stock Position in Union Pacific Co. (NYSE:UNP)
...
- **Current Stock Price of AAPL:** $226.80
- **GOOGL's Stock Price Comparison:** Today's price is $167.06, which is higher than the average price of $162.00 over the last 50 days in a 3-month period.

> Finished chain.
"Here is the information you requested:\n\n- **Current Date and Time:** 2024-10-05 18:47:35\n- **Weather in Calgary:** Scattered clouds\n- **Weather in Rome:** Clear sky\n- ** Scattered clouds\n- **Weather in Rome:** Clear sky\n- **Top 2 Stock Market News:**\n  1. **Appleton Partners Inc. MA Decreases Stock Holdings in iShares Core S&P Mid-Cap ETF (NYSEARCA:IJH)** - Appleton Partners Inc. MA lessened its stake in iShares Core S&P Mid-Cap ETF by 2.4% during the 3rd quarter. [Read more](https://www.etfdailynews.com/2024/10/04/appleton-partners-inc-ma-decreases-stock-holdings-in-ishares-core-sp-mid-cap-etf-nysearcaijh/)\n  2. **Appleton Partners Inc. MA Decreases Stock Position in Union Pacific Co. (NYSE:UNP)** - Appleton Partners Inc. MA lowered its position in shares of Union Pacific Co. by 2.4% during the third quarter. [Read more](https://www.etfdailynews.com/2024/10/04/appleton-partners-inc-ma-decreases-stock-position-in-union-pacific-co-nyseunp/)\n- **Current Stock Price of AAPL:** $226.80\n- **GOOGL's Stock Price Comparison:** Today's price is $167.06, which is higher than the average price of $162.00 over the last 50 days in a 3-month period."


## Results and Outputs
The AI agent effectively consolidates various types of information into a single interface, demonstrating its utility and practicality. For example, it can provide:

* Real-time weather updates for Calgary and Rome.
* Current stock prices for Apple and Google, along with a comparison to their
 three-month averages.
* The top two news headlines related to the stock market, helping users stay informed about recent developments.

``` terminal
> Finished chain.
"Here is the information you requested:

- **Current Date and Time:** 2024-10-05 18:47:35
- **Weather in Calgary:** Scattered clouds
- **Weather in Rome:** Clear sky
- **Top 2 Stock Market News:**
  1. **Appleton Partners Inc. MA Decreases Stock Holdings in iShares Core S&P Mid-Cap ETF (NYSEARCA:IJH)** - Appleton Partners Inc. MA lessened its stake in iShares Core S&P Mid-Cap ETF by 2.4% during the 3rd quarter. [Read more]
  (https://www.etfdailynews.com/2024/10/04/appleton-partners-inc-ma-decreases-stock-holdings-in-ishares-core-sp-mid-cap-etf-nysearcaijh/)
  2. **Appleton Partners Inc. MA Decreases Stock Position in Union Pacific Co. (NYSE:UNP)** - Appleton Partners Inc. MA lowered its position in shares of Union Pacific Co. by 2.4% during the third quarter. [Read more](https://www.etfdailynews.com/2024/10/04/appleton-partners-inc-ma-decreases-stock-position-in-union-pacific-co-nyseunp/)
- **Current Stock Price of AAPL:** $226.80
- **GOOGL's Stock Price Comparison:** Today's price is $167.06, which is higher than the average price of $162.00 over the last 50 days in a 3-month period."


## Conclusion
The developed AI agent serves as an efficient solution for investors by providing timely and accurate information on stock market trends, weather conditions, and significant daily news related to the market. With its flexible architecture and customizable features, this agent enables users to make more informed decisions. By seamlessly integrating live data and advanced analytics, this tool holds substantial potential for financial applications and can offer even greater value to users in the future by incorporating new features, such as predictive modeling and integration with trading platforms.

## Future Work
Future iterations of the AI agent could include advanced analytics features, such as predictive modeling for stock trends, integration with trading platforms for automated trading, and enhanced user interfaces for better engagement.

## Discussion
The design of this AI agent illustrates the potential to extend and customize its functionality according to specific client needs. By leveraging a multi-agent design, it can facilitate more complex analyses and decision-making processes, making it a powerful tool for investors.

## References
* OpenWeatherMap API documentation
* NewsAPI documentation
* Relevant financial data APIs

